<a href="https://colab.research.google.com/github/mukthar56788/Human-Body-Measurements-using-Computer-Vision/blob/master/lab9_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Load CIFAR-10 Dataset
def load_cifar10():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    train_data = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
    test_data = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

    train_loader = DataLoader(train_data, batch_size=len(train_data))
    test_loader = DataLoader(test_data, batch_size=len(test_data))

    X_train, y_train = next(iter(train_loader))
    X_test, y_test = next(iter(test_loader))

    X_train = X_train.view(X_train.size(0), -1).numpy()
    X_test = X_test.view(X_test.size(0), -1).numpy()

    y_train = y_train.numpy()
    y_test = y_test.numpy()

    return X_train, y_train, X_test, y_test

# (i) KNN Classifier
def knn_classifier(X_train, y_train, X_test, y_test, k=5):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    accuracy = knn.score(X_test, y_test)
    print(f"KNN Accuracy: {accuracy * 100:.2f}%")

# (ii) 3-Layer Neural Network
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 512)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(512, 256)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

def train_nn(model, train_loader, test_loader, epochs=10, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for images, labels in train_loader:
            images = images.view(images.size(0), -1)
            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {total_loss / len(train_loader):.4f}")

    # Evaluate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.view(images.size(0), -1)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Neural Network Accuracy: {100 * correct / total:.2f}%")

if __name__ == "__main__":
    # Load CIFAR-10 dataset
    X_train, y_train, X_test, y_test = load_cifar10()

    # (i) KNN Classifier
    knn_classifier(X_train, y_train, X_test, y_test, k=5)

    # (ii) 3-Layer Neural Network
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    train_data = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
    test_data = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

    train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

    model = NeuralNetwork()
    train_nn(model, train_loader, test_loader, epochs=10, learning_rate=0.001)


100%|██████████| 170M/170M [00:01<00:00, 90.7MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
KNN Accuracy: 33.98%
Files already downloaded and verified
Files already downloaded and verified
Epoch [1/10], Loss: 1.6321
Epoch [2/10], Loss: 1.4234
Epoch [3/10], Loss: 1.3112
Epoch [4/10], Loss: 1.2178
Epoch [5/10], Loss: 1.1361
Epoch [6/10], Loss: 1.0597
Epoch [7/10], Loss: 0.9828
Epoch [8/10], Loss: 0.9098
Epoch [9/10], Loss: 0.8407
Epoch [10/10], Loss: 0.7673
Neural Network Accuracy: 53.23%
